In [8]:
import robotic as ry
import numpy as np
import time

In [9]:
config = ry.Config()
config.addFile("puzzles/p5-wall-easy.g")

In [10]:
print(config.getFrameNames())

['world', 'floor', 'outwall_right', 'outwall_back', 'outwall_left', 'outwall_front', 'sub-goal1', 'world', 'floor', 'outwall_right', 'outwall_back', 'outwall_left', 'outwall_front', 'sub-goal1', 'camera_top', 'egoJoint', 'ego', 'goal', 'goal_visible', 'wall2_h', 'wall8_h', 'obj', 'goalLarge']


In [11]:
EGO_NAME = "ego"
OBJ_NAME = "obj"
SUB_GOAL_NAME = "sub-goal1"
GOAL_NAME = "goal"
CAMERA_NAME = "camera_top"
FLOOR_NAME = "floor"

GAP = config.getFrame(EGO_NAME).getSize()[1] * 3

In [12]:
config.view() # somehow it breaks if I don't call this before setting the camera
f = config.getFrame('camera_top')
config.view_setCamera(f)

In [6]:
q_start = config.getFrame(EGO_NAME).getPosition()[:2]
q_obj = config.getFrame(OBJ_NAME).getPosition()[:2]
q_goal = config.getFrame(GOAL_NAME).getPosition()[:2]

In [7]:
komo = ry.KOMO(config, phases=1, slicesPerPhase=1, kOrder=1, enableCollisions=True)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, scale=[1e2])
komo.addObjective([], ry.FS.negDistance, [EGO_NAME, OBJ_NAME], ry.OT.eq, [1e2])

ret = ry.NLP_Solver(komo.nlp(), verbose=3).solve()
print(ret)

====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1 	lambda:[]
{ time: 0.00362, evals: 8, done: 1, feasible: 0, sos: 0, f: 0, ineq: 0, eq: 20 }
----newton---- initial point f(x):18115.1 alpha:1 beta:1
--newton-- it:   1  |Delta|:        0.2  alpha:          1  evals:   2  f(y):    12405.1  ACCEPT
--newton-- it:   2  |Delta|:        0.2  alpha:          1  evals:   3  f(y):    7772.83  ACCEPT
--newton-- it:   3  |Delta|:        0.2  alpha:          1  evals:   4  f(y):    4218.41  ACCEPT
--newton-- it:   4  |Delta|:        0.2  alpha:          1  evals:   5  f(y):     1741.8  ACCEPT
--newton-- it:   5  |Delta|:        0.2  alpha:          1  evals:   6  f(y):    378.626  ACCEPT
--newton-- it:   6  |Delta|:        0.2  alpha:          1  evals:   7  f(y):    286.912  ACCEPT
--newton-- it:   7  |Delta|:  0.0659197  alpha:          1  evals:   8  f(y):        200  ACCEPT
--newton-- it:   8  |Delta|:1.64795e-06 	 -- absMax(Delta)<1e-1*o.stopTolerance 

In [58]:
config.setJointState(q_start)
config.view()
print(ret.x)

[-0.78454607 -1.14999977]


In [57]:
for state in komo.getPath():
    config.setJointState(state)
    config.view()
    time.sleep(1)

In [38]:
komo = ry.KOMO(config, phases=2, slicesPerPhase=1, kOrder=1, enableCollisions=True)
komo.addQuaternionNorms() # what is this?
komo.addModeSwitch([1.,-1.], ry.SY.stable, [EGO_NAME, OBJ_NAME], True)
komo.addObjective([1.], ry.FS.negDistance, [EGO_NAME, OBJ_NAME], ry.OT.eq, [1e1])
komo.addObjective([2.], ry.FS.positionDiff, [OBJ_NAME, GOAL_NAME], ry.OT.eq, [1e1])

ret = ry.NLP_Solver(komo.nlp(), verbose=3).solve()
print(ret)

{ time: 0.006769, evals: 15, done: 1, feasible: 0, sos: 0, f: 0, ineq: 0, eq: 1.50214 }
====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1
----newton---- initial point f(x):795.56 alpha:1 beta:1
--newton-- it:   1  |Delta|:        0.2  alpha:          1  evals:   2  f(y):    685.676  ACCEPT
--newton-- it:   2  |Delta|:        0.2  alpha:          1  evals:   3  f(y):    583.975  ACCEPT
--newton-- it:   3  |Delta|:        0.2  alpha:          1  evals:   4  f(y):    490.481  ACCEPT
--newton-- it:   4  |Delta|:        0.2  alpha:          1  evals:   5  f(y):    405.188  ACCEPT
--newton-- it:   5  |Delta|:        0.2  alpha:          1  evals:   6  f(y):    327.391  ACCEPT
--newton-- it:   6  |Delta|:        0.2  alpha:          1  evals:   7  f(y):    257.158  ACCEPT
--newton-- it:   7  |Delta|:        0.2  alpha:          1  evals:   8  f(y):    195.466  ACCEPT
--newton-- it:   8  |Delta|:        0.2  alpha:          1  evals:   9  f(y):      14

In [39]:
print(config.getCollisionFree())

False


In [40]:
# config.view_close()
# del config